# etudehouse

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import time
import copy
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def getNumber(string):
    numExtracter = re.compile('[0-9]+')
    return int(''.join(numExtracter.findall(string)))
    

In [3]:
def getCategoryList(url_products):
    driver.get(url_products)
    html = driver.page_source
    soup = bs(html,'html.parser')
    categories = soup.find('dl',{'class':'focus_target'}).dd.ul.find_all('li')
    categoryList = [url_home + category.a['href'] for category in categories]
    categoryNameList = [category.a.get_text() for category in categories]
    return categoryList, categoryNameList


In [4]:
def clickNextPage():
    html = driver.page_source
    soup = bs(html,'html.parser')
    nextPage = soup.find('a',{'class':'navi next'})
    if nextPage:
        driver.find_element_by_link_text('next page').click()
    else:
        raise

In [5]:
def getItemList():
    html = driver.page_source
    soup = bs(html,'html.parser')
    items = soup.find('div',{'class':'item_list column2'}).find_all('div',{'class':'item item-apply'})
    itemList = [url_home + item.find('div',{'class':'item_images'}).a['href'] for item in items]
    return itemList

In [15]:
def getItem(categoryName):
    html = driver.page_source
    soup = bs(html, 'html.parser')

    name = soup.find('h3',{'class':'h_title'}).get_text().strip()
    images = soup.find('div',{'class':'prd_thm_wrap preview_thumbs'}).find_all('img')
    imageList = [image['src'] for image in images]
    category = categoryName
    volume = soup.find('th',text='용량')
    volume = volume.parent.td.get_text() if volume else '#'
    originalPrice = getNumber(soup.find('span',{'class':'price'}).get_text())
    salePrice = originalPrice
    brand = 'etudehouse'
    url = driver.current_url
    colors = soup.find('button',text='옵션선택')
    if colors: 
        image_colors = colors.parent.find_all('img')
        image_colorList = [image_color['src'] for image_color in image_colors] if image_colors else []
    colors = colors.parent.ul.find_all('code') if colors else '#'
    colorList = [color.get_text().strip() for color in colors] if colors != '#' else '#'
    pd_type = soup.find('th',text='제품 주요 사양 - 피부타입, 색상(호,번)등')
    pd_type = pd_type.parent.td.get_text() if pd_type else '#'
    
    
    item = {'name':'#', 'url':'#', 'image':'#', 'color':'#', 'category':'#', 
                   'salePrice':'#', 'originalPrice':'#', 'brand':'#','volume':'#','type':'#'}
    item['name']=name
    item['image']=imageList
    item['category']=category
    item['volume']=volume
    item['originalPrice']=originalPrice
    item['salePrice']=salePrice
    item['brand']=brand
    item['url']=url
    item['type']=pd_type

    items = []
    if colorList != '#':
        for co, color in enumerate(colorList):
            item_copy = copy.deepcopy(item)
            item_copy['color'] = color
            if image_colorList:
                item_copy['image'].append(image_colorList[co])
            items.append(item_copy)
    else:
        items.append(item)
    print('상품명 :',item['name'])
    print('색상 수 :', len(items))
    return items

In [7]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


In [16]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

url_home = 'https://www.etudehouse.com'
url_products = 'https://www.etudehouse.com/kr/ko/main'

In [17]:
start_time = time.time() 
#---------------------------------------------------------------------#
driver.get(url_products)
categoryList, categoryNameList = getCategoryList(url_products)
result = []
for c, category in enumerate(categoryList):
    driver.get(category)
    categoryName = categoryNameList[c]
    itemList = []
    while True:
        try:
            newItemList = getItemList()
            if newItemList[0] not in itemList:
                itemList += newItemList
                clickNextPage()
            else:
                time.sleep(2)
                continue
            
        except:
            break
            
    print('상품 개수 :', len(list(set(itemList))))
    for item in itemList:
        driver.get(item)
        result += getItem(categoryName)
    
#---------------------------------------------------------------------#
print("--- %0.2f seconds ---" %(time.time() - start_time))

상품 개수 : 79
상품명 : 플레이 컬러 아이즈 미니 키트
색상 수 : 2
상품명 : 이지 핏 젤 아이라이너
색상 수 : 3
상품명 : 플레이 컬러 아이즈 라벤더 랜드
색상 수 : 1
상품명 : 해피 위드 피글렛 룩 앳 마이 아이즈
색상 수 : 10
상품명 : 플레이 101 펜슬 NEW
색상 수 : 35
상품명 : 퀵앤이지 매트&글리터 리퀴드 섀도우
색상 수 : 5
상품명 : 퀵앤이지 피니쉬 섀도우&라이너
색상 수 : 5
상품명 : 타이니 트윙클 컬러 아이즈
색상 수 : 3
상품명 : 플레이 컬러 아이 팔레트 #베스트 러브드
색상 수 : 1
상품명 : 플레이 컬러 아이즈 카페인 홀릭
색상 수 : 1
상품명 : 청순거짓 브라우 볼류마이저
색상 수 : 6
상품명 : 룩 앳 마이 아이즈 너츠 앤 후르츠
색상 수 : 7
상품명 : 오 마이 래쉬 마스카라 NEW
색상 수 : 4
상품명 : 닥터 마스카라 컬러 픽서
색상 수 : 2
상품명 : 드로잉 아이브라우 프루프 젤 펜슬
색상 수 : 6
상품명 : 반짝 눈물 라이너
색상 수 : 3
상품명 : 마이래쉬세럼
색상 수 : 1
상품명 : 룩 앳 마이 아이즈 쥬얼
색상 수 : 42
상품명 : 블렌드 포 아이즈
색상 수 : 7
상품명 : 블링블링 아이스틱
색상 수 : 8
상품명 : 듀얼 트임 마스카라
색상 수 : 5
상품명 : 액티브 프루프 브라우&헤어 드로잉 팁
색상 수 : 2
상품명 : 플레이 컬러 멀티 팔레트
색상 수 : 3
상품명 : 플레이 컬러 아이 팔레트
색상 수 : 3
상품명 : 룩 앳 마이 아이즈 젤라또
색상 수 : 5
상품명 : 미러 홀릭 리퀴드 아이즈
색상 수 : 10
상품명 : 프루프 10 젤 방수 펜슬
색상 수 : 6
상품명 : 룩 앳 마이 아이즈 카페
색상 수 : 34
상품명 : 플레이 컬러 아이즈 피치팜
색상 수 : 1
상품명 : 룩 앳 마이 아이즈
색상 수 : 79
상품명 : 컬러풀 드로잉 슈퍼 슬림 프루프 젤 펜슬 라이너
색상 수 : 1
상품명 : 컬러풀 드로잉 판타스틱 컬러 아이즈
색상 수 : 2

상품명 : 모던 옴므 부스팅 토너
색상 수 : 1
상품명 : 순정 스킨케어 3종 세트
색상 수 : 1
상품명 : 젠틀블랙 프레시 토너
색상 수 : 1
상품명 : 젠틀블랙 하이드레이팅 에멀전
색상 수 : 1
상품명 : 원더포어 프레쉬너 블랙 500ml
색상 수 : 1
상품명 : 순정 수분 진정 올인원 젤 120ml
색상 수 : 1
상품명 : 액티브 프루프 아쿠아 쿨링 선 워터 SPF50+/PA++++
색상 수 : 1
상품명 : 액티브 프루프 실키 글라이드 선 스틱 SPF50+/PA++++
색상 수 : 1
상품명 : 액티브 프루프 쿨 다운 수딩 젤
색상 수 : 1
상품명 : 펩타이드 버블 업 세럼
색상 수 : 1
상품명 : 순정 스킨케어 2종세트
색상 수 : 1
상품명 : 유브이 더블 컷 산뜻 선 젤 SPF50+/PA++++
색상 수 : 2
상품명 : 워터 멜팅 산뜻 선 스틱 SPF50+ PA++++
색상 수 : 1
상품명 : 유브이 더블 컷 투명 선 스틱 SPF50+/PA++++
색상 수 : 1
상품명 : 유브이 더블 컷 촉촉 톤업  선 플루이드 SPF48/PA+++
색상 수 : 1
상품명 : 순정 진정 방어 선크림 SPF49/PA++
색상 수 : 2
상품명 : 올 케어링 이레이저 패드
색상 수 : 1
상품명 : 선프라이즈 내추럴 코렉터 라이트 SPF50+/PA++++
색상 수 : 1
상품명 : 순정 약산성 5.5 진정 토너
색상 수 : 3
상품명 : 순정 2x 베리어 보습 크림
색상 수 : 2
상품명 : 순정 판텐소사이드™ 5 시카 밤
색상 수 : 2
상품명 : 선프라이즈 내추럴 코렉터 SPF50+/PA+++
색상 수 : 1
상품명 : 수분가득 콜라겐 블렌딩 딥 에센스
색상 수 : 1
상품명 : 지니 멀티 오일
색상 수 : 1
상품명 : 지니 멀티 밤
색상 수 : 1
상품명 : 발그레 수딩 & 수분 쏙 밸런싱 올인원 젤
색상 수 : 2
상품명 : DIY 트래블 키트
색상 수 : 11
상품명 : 원더포어 타이트닝 에센스 50ml
색상 수 : 1
상품명 : 원더포

상품명 : 9LOOKS 스페셜 파우치
색상 수 : 1
상품명 : 마이뷰티툴 퍼스널 브라우 밴드
색상 수 : 2
상품명 : 마이뷰티툴 각질정리해-파리 실리콘 브러쉬
색상 수 : 1
상품명 : 마이뷰티툴 띠부띠부 스티커 속눈썹
색상 수 : 3
상품명 : 섀도우 공용기 4구
색상 수 : 5
상품명 : 슈가 앤 잼 LED 거울
색상 수 : 1
상품명 : 마이뷰티툴 브러쉬 312 섀도우 포인트
색상 수 : 1
상품명 : 마이뷰티툴 브러쉬 311 섀도우 블랜딩
색상 수 : 1
상품명 : 마이뷰티툴 브러쉬 310 섀도우 베이스
색상 수 : 1
상품명 : 플레이 101 스틱 픽스&블렌딩 브러쉬
색상 수 : 1
상품명 : 순정 드라이 캡슐 시트 5매입
색상 수 : 1
상품명 : 마이뷰티툴 양면 퍼프
색상 수 : 1
상품명 : 원더 펀 파크 캔디 치크 브러쉬
색상 수 : 1
상품명 : 원더 펀 파크 컬러 아이즈 섀도우 브러쉬
색상 수 : 1
상품명 : 컬러풀 센트 퍼퓸 향수 공용기
색상 수 : 1
상품명 : 마이뷰티툴 섀도우 공용기
색상 수 : 3
상품명 : 애니쿠션 브러쉬 1호
색상 수 : 1
상품명 : 플레이 101 블렌딩 펜슬 핑거 브러쉬
색상 수 : 1
상품명 : 마이뷰티툴 웨지 퍼프
색상 수 : 1
상품명 : 마이뷰티툴 버블 메이커
색상 수 : 1
상품명 : 더블래스팅 파운데이션 블렌더
색상 수 : 1
상품명 : 슈가 앤 잼 스윗 에나멜 파우치 블랙
색상 수 : 1
상품명 : 슈가 앤 잼 스윗 에나멜 파우치 핑크
색상 수 : 1
상품명 : 마이뷰티툴 브러쉬 150 블러쉬&컨투어
색상 수 : 1
상품명 : 더블래스팅 파운데이션 브러쉬
색상 수 : 1
상품명 : 플레이 101 스틱_브러쉬
색상 수 : 1
상품명 : 마이뷰티툴 족집게
색상 수 : 1
상품명 : 마이뷰티툴 마이크로 극세사 퍼프 2p
색상 수 : 1
상품명 : 마이뷰티툴 속눈썹 눈꼬리 & 언더라인
색상 수 : 1
상품명 : 마이뷰티툴 미용 가위
색상 수 : 1
상품명 : 마이뷰티툴 속눈썹 3단계 & 4단계
색상

In [18]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'etudehouse.json')